In [1]:
from ccdproc import CCDData
from astropy.io import fits, registry
from astropy import units as u
from astropy.table import Table

In [2]:
class MOSFIREimage(CCDData):
    '''Subclass of ccdproc.CCDData to hold a MOSFIRE image'''
    def __init__(self, *args, **kwd):
        super(MOSFIREimage, self).__init__(*args, **kwd)
        self.target_list = None
        self.ssl = None
        self.msl = None
        self.asl = None
        self.target_table = None
        self.science_slits = None
        self.mech_slits = None
        self.alignment_slits = None

def fits_mosfire_reader(filename, unit='adu'):
    '''Read a MOSFIRE multi-extension fits in to a MOSFIREimage object.    
    '''
    hdulist = fits.open(filename, 'readonly')
    image = MOSFIREimage(hdulist[0].data, meta=hdulist[0].header, unit=unit)
    image.target_list = hdulist[1].data
    image.target_table = Table(image.target_list)
    image.ssl = hdulist[2].data
    image.science_slits = Table(image.ssl)
    image.msl = hdulist[3].data
    image.mech_slits = Table(image.msl)
    image.asl = hdulist[4].data
    image.alignment_slits = Table(image.asl)

    return image

registry.register_reader('fits', MOSFIREimage, fits_mosfire_reader)
# registry.register_writer('fits', MOSFIREimage, fits_mosfire_writer)


In [3]:
filename = '/Volumes/Internal_1TB/MOSFIRE_Data/TestCase/DRP_Test_Case_Hband/2012sep10/m120910_0410.fits'
im = MOSFIREimage.read(filename, unit='adu')

In [4]:
# The data is an ndarray
print(type(im.data))
im.data

<type 'numpy.ndarray'>


array([[ -9.5 , -15.5 , -12.5 , ...,  -2.5 ,   5.5 ,  12.5 ],
       [-11.5 ,  -5.5 , -14.5 , ...,   4.5 ,  20.5 ,   2.5 ],
       [-26.5 ,   2.5 , -20.5 , ...,  23.5 ,  10.5 , -18.5 ],
       ..., 
       [ -0.25,  -9.25,   3.75, ...,  13.75,   9.75,   6.75],
       [-24.25,  -2.25,  -8.25, ...,   7.75,  -2.25, -10.25],
       [ -0.25,   6.75,  -2.25, ...,  -6.25,  18.75,  19.75]], dtype=float32)

In [5]:
# The target list is accessible as part of the im object as a FITS record (as it was before)
im.target_list

FITS_rec([ ('TARG24', '100.00', '15.95', '22', '59', '41.38', '33', '0', '14.10', '2000.0', '2000.0'),
       ('TARG7', '100.00', '15.41', '22', '59', '45.31', '33', '2', '17.80', '2000.0', '2000.0'),
       ('TARG25', '100.00', '15.95', '22', '59', '48.11', '33', '0', '10.40', '2000.0', '2000.0'),
       ('TARG11', '100.00', '15.62', '22', '59', '50.64', '32', '59', '21.60', '2000.0', '2000.0'),
       ('TARG23', '100.00', '15.94', '22', '59', '53.98', '32', '59', '39.40', '2000.0', '2000.0'),
       ('TARG19', '100.00', '15.86', '22', '59', '56.56', '33', '0', '30.20', '2000.0', '2000.0'),
       ('TARG14', '100.00', '15.69', '22', '59', '57.28', '32', '59', '24.20', '2000.0', '2000.0'),
       ('TARG12', '100.00', '15.65', '23', '0', '0.38', '33', '0', '25.30', '2000.0', '2000.0'),
       ('TARG20', '100.00', '15.89', '23', '0', '2.64', '33', '0', '58.70', '2000.0', '2000.0'),
       ('TARG6', '100.00', '15.38', '23', '0', '5.53', '33', '0', '40.90', '2000.0', '2000.0'),
       ('TA

In [6]:
# The target list is also accessible in the more human readable astropy.table.Table format.
im.target_table

Target_Name,Priority,Magnitude,RA_Hours,RA_Minutes,RA_Seconds,Dec_Degrees,Dec_Minutes,Dec_Seconds,Epoch,Equinox
str6,str6,str5,str2,str2,str5,str2,str2,str5,str6,str6
TARG24,100.00,15.95,22,59,41.38,33,0,14.10,2000.0,2000.0
TARG7,100.00,15.41,22,59,45.31,33,2,17.80,2000.0,2000.0
TARG25,100.00,15.95,22,59,48.11,33,0,10.40,2000.0,2000.0
TARG11,100.00,15.62,22,59,50.64,32,59,21.60,2000.0,2000.0
TARG23,100.00,15.94,22,59,53.98,32,59,39.40,2000.0,2000.0
TARG19,100.00,15.86,22,59,56.56,33,0,30.20,2000.0,2000.0
TARG14,100.00,15.69,22,59,57.28,32,59,24.20,2000.0,2000.0
TARG12,100.00,15.65,23,0,0.38,33,0,25.30,2000.0,2000.0
TARG20,100.00,15.89,23,0,2.64,33,0,58.70,2000.0,2000.0


In [7]:
# The science slits are available as both a FITS record and an astropy Table
im.science_slits

Slit_Number,Slit_RA_Hours,Slit_RA_Minutes,Slit_RA_Seconds,Slit_Dec_Degrees,Slit_Dec_Minutes,Slit_Dec_Seconds,Slit_width,Slit_length,Target_to_center_of_slit_distance,Target_Name,Target_Priority
str2,str2,str2,str5,str2,str2,str5,str5,str6,str7,str6,str6
1,22,59,41.43,33,0,13.88,0.700,14.990,0.680,TARG24,100.00
2,22,59,45.68,33,2,16.20,0.700,22.970,4.900,TARG7,100.00
3,22,59,46.74,33,0,16.35,0.700,62.860,-18.270,TARG25,100.00
4,22,59,50.21,32,59,23.48,0.700,46.900,-5.770,TARG11,100.00
5,22,59,53.54,32,59,41.30,0.700,22.970,-5.840,TARG23,100.00
6,22,59,56.85,33,0,28.93,0.700,30.950,3.900,TARG19,100.00
7,22,59,57.43,32,59,23.55,0.700,14.990,2.000,TARG14,100.00
8,23,0,0.39,33,0,25.27,0.700,22.970,0.090,TARG12,100.00
9,23,0,3.03,33,0,57.00,0.700,22.970,5.220,TARG20,100.00


In [8]:
# The mechanical slits are available as both a FITS record and an astropy Table
im.mech_slits

Slit_Number,Target_in_Slit,Target_Priority,Position_of_Slit,Slit_width,Target_to_center_of_slit_distance
str2,str6,str6,str7,str5,str5
1,TARG24,100.00,-49.538,0.700,0.000
2,TARG24,100.00,-50.096,0.700,0.000
3,TARG7,100.00,82.727,0.700,0.000
4,TARG7,100.00,82.169,0.700,0.000
5,TARG7,100.00,81.611,0.700,0.000
6,TARG25,100.00,-28.205,0.700,0.000
7,TARG25,100.00,-28.763,0.700,0.000
8,TARG25,100.00,-29.320,0.700,0.000
9,TARG25,100.00,-29.878,0.700,0.000


In [9]:
# The alignment slits are available as both a FITS record and an astropy Table
im.alignment_slits

Slit_Number,Position_of_Slit,Slit_width,Target_to_center_of_slit_distance,Target_in_Slit,Target_Priority,Target_Magnitude,Target_RA_Hours,Target_RA_Minutes,Target_RA_Seconds,Target_Dec_Degrees,Target_Dec_Minutes,Target_Dec_Seconds,Target_Epoch,Target_Equinox
str2,str7,str5,str6,str3,str5,str5,str2,str2,str5,str2,str2,str5,str6,str6
3,70.428,4.000,-1.010,S14,-1.00,15.45,22,59,44.90,33,2,6.60,2000.0,2000.0
15,96.160,4.000,0.600,S3,-1.00,14.32,22,59,52.66,33,2,7.20,2000.0,2000.0
22,28.025,4.000,-0.120,S7,-1.00,14.72,22,59,55.60,33,0,46.70,2000.0,2000.0
24,-68.463,4.000,0.280,S15,-1.00,15.48,22,59,54.81,32,59,9.50,2000.0,2000.0
27,81.849,4.000,1.220,S10,-1.00,15.04,22,59,59.67,33,1,28.80,2000.0,2000.0
42,-36.552,4.000,0.730,S4,-1.00,14.45,23,0,6.46,32,59,3.30,2000.0,2000.0


In [10]:
filename2 = '/Volumes/Internal_1TB/MOSFIRE_Data/TestCase/DRP_Test_Case_Hband/2012sep10/m120910_0411.fits'
im2 = MOSFIREimage.read(filename2, unit='adu')

In [11]:
sum = im.add(im2)

In [12]:
#print out mean levels to confirm sum
print(im.data.mean())
print(im2.data.mean())
print(sum.data.mean())

14858.5
16313.3
31171.9


In [13]:
diff = im.subtract(im2)
print(diff.data.mean())

-1454.8
